In [1]:
import ee
import geemap
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import zipfile
from osgeo import gdal
from osgeo import ogr
from osgeo import gdalconst

In [2]:
# Pre-Processing
def preProc(in_fn, out_fn, crs):
    gdf = gpd.read_file(in_fn)
    gdf['ID'] = 1
    gdf.geometry = gdf.make_valid()
    gdf = gdf[['ID','geometry']]
    gdf = gdf.dissolve()
    
    gdf.to_crs(crs).to_file(out_fn, driver = 'GeoJSON')

In [3]:
preProc('../Data/Vector/Cashew_Tanzania.shp', '../Data/Vector/Cashew_Tanz.geojson', 'epsg:32736') # Tanzania
preProc('../Data/Vector/Cashew_Ivorycoast.shp', '../Data/Vector/Cashew_Ivy.geojson', 'epsg:2041') # IvyCst

In [4]:
def rasterize_shp(base, shp, output):
    data = gdal.Open(base, gdalconst.GA_ReadOnly)
    geo_transform = data.GetGeoTransform()
    
    x_min = geo_transform[0]
    y_max = geo_transform[3]
    x_max = x_min + geo_transform[1] * data.RasterXSize
    y_min = y_max + geo_transform[5] * data.RasterYSize
    x_res = data.RasterXSize
    y_res = data.RasterYSize
    
    vec = ogr.Open(shp)
    lyr = vec.GetLayer()
    pixel_width = geo_transform[1]
    target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
    target_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
    
    band = target_ds.GetRasterBand(1)
    band.FlushCache()
    
    gdal.RasterizeLayer(target_ds, [1], lyr, burn_values = [1])
    
    target_ds = None

In [10]:
base = '../Data/Planet/TanzaniaStudyArea.tif'
shp = '../Data/Vector/Cashew_Tanz.geojson'
output = '../Data/Vector/TanzaniaRasterized.tif'
rasterize_shp(base, shp, output)

In [5]:
for i in os.listdir('../Data/Planet'):
    if 'PlanetTilesIvory' in i:
        base = '../Data/Planet/' + i
        shp = '../Data/Vector/Cashew_Ivy.geojson'
        output = '../Data/Vector/IvoryCoastRasterized_'+i[-5:-4]+'.tif'
        rasterize_shp(base, shp, output)